<a href="https://colab.research.google.com/github/leandroteleco/meraki-auto/blob/main/Meraki_auto_TWO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalación de librerías meraki y openpyxl

In [7]:
# Install the relevant modules. If you are using a local editor (e.g. VS Code, rather than Colab) you can run these commands, without the preceding %, via a terminal. NB: Run `pip install meraki==` to find the latest version of the Meraki SDK. Uncomment these lines to run them in Google Colab.
%pip install meraki
%pip install openpyxl

In [11]:
# If you are using Google Colab, please ensure you have set up your environment variables as linked above, then delete the two lines of ''' to activate the following code:
%pip install colab-env -qU
import colab_env


# The Meraki SDK
import meraki
# The built-in OS module, to read environment variables
import os
# We're also going to import Python's built-in JSON module, but only to make the console output pretty. In production, you wouldn't need any of the printing calls at all, nor this import!
import json

# The openpyxl module, to manipulate Excel files
import openpyxl
# The datetime module, to generate timestamps
import datetime

# Treat your API key like a password. Store it in your environment variables as 'MERAKI_DASHBOARD_API_KEY' and let the SDK call it for you.
# Or, call it manually after importing Python's os module:
# API_KEY = os.getenv('MERAKI_DASHBOARD_API_KEY')
from google.colab import userdata
API_KEY = userdata.get('MERAKI_DASHBOARD_API_KEY')
JOB_ORDER_EXCEL_NAME = 'New-SSID-Merlin.xlsx'

# Initialize the Dashboard connection.
dashboard = meraki.DashboardAPI(API_KEY, suppress_logging=True)

# We'll also create a few reusable strings for later interactivity.
string_constants = dict()
string_constants['CONFIRM'] = 'OK, are you sure you want to do this? This script does not have an "undo" feature.'
string_constants['CANCEL'] = 'OK. Operation canceled.'
string_constants['WORKING'] = 'Working...'
string_constants['COMPLETE'] = 'Operation complete.'
string_constants['NETWORK_SELECTED'] = 'Network selected.'
string_constants['NO_VALID_OPTIONS'] = 'There are no valid options. Please try again with an API key that has access to the appropriate resources.'

# Some of the parameters we'll work with are optional. This string defines what value will be put into a cell corresponding with a parameter that is not set on that rule.
string_constants['NOT_APPLICABLE'] = 'N/A'

# This script is interactive; user choices and data will be stored here.
user_choices = dict()
user_data = dict()

def printj(ugly_json_object):

     # The json.dumps() method converts a JSON object into human-friendly formatted text
    pretty_json_string = json.dumps(ugly_json_object, indent = 2, sort_keys = False)

    return print(pretty_json_string)

class UserChoice:

    'A re-usable CLI option prompt.'

    def __init__(self, options_list=[], subject_of_choice='available options', single_option_noun='option', id_parameter='id', name_parameter='name', action_verb='choose', no_valid_options_message=string_constants['NO_VALID_OPTIONS']):

        self.options_list = options_list # options_list is a list of dictionaries containing attributes id_parameter and name_parameter
        self.subject_of_choice = subject_of_choice # subject_of_choice is a string that names the subject of the user's choice. It is typically a plural noun.
        self.single_option_noun = single_option_noun # single_option_noun is a string that is a singular noun corresponding to the subject_of_choice
        self.id_parameter = id_parameter # id_parameter is a string that represents the name of the sub-parameter that serves as the ID value for the option in options_list. It should be a unique value for usability.
        self.name_parameter = name_parameter # name_paraemter is a string that represents the name of the sub-parameter that serves as the name value for the option in options_list. It does not need to be unique.
        self.action_verb = action_verb # action_verb is a string that represents the verb of the user's action. For example, to "choose"
        self.no_valid_options_message = no_valid_options_message # no_valid_options_message is a string that represents an error message if options_list is empty

        # Confirm there are options in the list
        if len(self.options_list):
            print(f'We found {len(self.options_list)} {self.subject_of_choice}:')

            # Label each option and show the user their choices.
            option_index = 0

            for option in self.options_list:
                print(f"{option_index}. {option[self.id_parameter]} with name {option[self.name_parameter]}")
                option_index+=1

            print(f'Which {self.single_option_noun} would you like to {self.action_verb}?')
            self.active_option = int(input(f'Choose 0-{option_index-1}:'))

            # Ask until the user provides valid input.
            while self.active_option not in list(range(option_index)):
                print(f'{self.active_option} is not a valid choice. Which {self.single_option_noun} would you like to {self.action_verb}?')
                self.active_option = int(input(f'Choose 0-{option_index-1}:'))

            print(f'Your {self.single_option_noun} is {self.options_list[self.active_option][self.name_parameter]}.')

            # Assign the class id and name vars to the chosen item's
            self.id = self.options_list[self.active_option][self.id_parameter]
            self.name = self.options_list[self.active_option][self.name_parameter]



def get_ssid_info(*, networkId):
    # Get the SSID Info
    ssid_info = dashboard.wireless.getNetworkWirelessSsids(networkId=networkId)

    return(ssid_info)

def create_workbook():

    # Create a workbook with the appropriate column headers and/or worksheets
    # First we'll create the workbook, then we'll design the worksheets. Just like in Excel, by default, the workbook has one worksheet, titled 'Sheet'.
    new_workbook = openpyxl.Workbook()

    # We'll specify that the active worksheet is our ssid_info_worksheet
    ssid_info_worksheet = new_workbook.active

    # Let's rename the worksheet from 'Sheet' to something more descriptive
    ssid_info_worksheet.title = 'SSID-Info'

    # Name the columns for the ssid_info_worksheet. Think of this like a single-line CSV:
    ssid_info_title_row_headers = (
        'SSID',
        'Visibility',
        'Access',
        'Security',
        'Password',
        'DNS',
        'FW Rule Plocy',
        'FW Rule Protocol',
        'FW Rule Destination',
        'FW Rule Port',
        'FW Rule Comment'
    )


    # Add that title row to the worksheet
    ssid_info_worksheet.append(ssid_info_title_row_headers)

    # Let's make the title row bold for easier reading
    for row in ssid_info_worksheet.iter_rows():
        for cell in row:
            cell.font = openpyxl.styles.Font(bold=True)

    # Now let's do the same for the Switch Ports Info.
    # First, create a separate worksheet for the Switch Ports Info
    switch_ports_info_worksheet = new_workbook.create_sheet(title='Switch-Ports-Info')

    # Name the columns for the vpn_prefs_worksheet and custom_performance_classes_worksheet.
    switch_ports_info_title_row_headers = (
        'Sede',
        'Switch',
        'Puerto físico',
        'Negociación',
        'Tipo',
        'Descripción',
        'VLAN',
        'Voice VLAN'
    )

    # Add the title rows to the appropriate worksheets
    switch_ports_info_worksheet.append(switch_ports_info_title_row_headers)

    # Let's make those title rows bold, too
    for row in switch_ports_info_worksheet.iter_rows():
        for cell in row:
            cell.font = openpyxl.styles.Font(bold=True)

    print(f'Created formatted workbook.')

    return(new_workbook)



def add_ssid_info_to_workbook(workbook, network_id):

    # We'll specify that the active worksheet is our ssid_info worksheet
    ssid_info_worksheet = workbook['SSID-Info']

    # We'll also count the number of SSID to help the user know that it's working.
    ssid_count = 0

    # Let's add all the SSID Info to the SSID-Info worksheet
    for ssid_info in user_data['ssid_info']:

        ssid_info_ssid_name = ssid_info['name']
        ssid_info_visibility = ssid_info['visible']
        ssid_info_access = ssid_info['encryptionMode']
        ssid_info_security = ssid_info['wpaEncryptionMode']
        ssid_info_password = ssid_info['psk']
        try:
          ssid_info_dns = ssid_info['dnsRewrite']['dnsCustomNameservers']

        fw_wireless_rules = dashboard.wireless.getNetworkWirelessSsidFirewallL3FirewallRules(network_id, ssid_count)
        ssid_info_fw_rule_policy = fw_wireless_rules['rules']['policy']
        ssid_info_fw_rule_protocol = fw_wireless_rules['rules']['protocol']
        ssid_info_fw_rule_destination = fw_wireless_rules['rules']['destCidr']
        ssid_info_fw_rule_port = fw_wireless_rules['rules']['destPort']
        ssid_info_fw_rule_comment = fw_wireless_rules['rules']['comment']

        # We assemble the parameters into a tuple that will represent a row.
        ssid_info_row = (
            ssid_info_ssid_name,
            ssid_info_visibility,
            ssid_info_access,
            ssid_info_security,
            ssid_info_password,
            ssid_info_dns,
            ssid_info_fw_rule_policy,
            ssid_info_fw_rule_protocol,
            ssid_info_fw_rule_destination,
            ssid_info_fw_rule_comment
            )

        # We then add that row to the worksheet.
        ssid_info_worksheet.append(ssid_info_row)

        # increase the ssid_count
        ssid_count += 1


    print(f'Added {ssid_count} performance classes to customPerformanceClasses.')

    return(workbook)


# Function that saves a workbook
def save_workbook(workbook):

    # Finally, we save the worksheet.
    # Let's give it a name with a date/time stamp
    downloaded_rules_workbook_filename = f'ssid_info_workbook_{datetime.datetime.now()}.xlsx'.replace(':','')

    workbook.save(downloaded_rules_workbook_filename)

    print(f'Saved {downloaded_rules_workbook_filename}.')

# Function that load a workbook template to read values to set/configure network.
def load_excel(excel_name):
    try:
        print ('Loading excel...')
        excel_document = openpyxl.load_workbook(excel_name)
        print(excel_document)
        print(excel_document.sheetnames)
        print (excel_document['SSID '].values)
        #for row in excel_document['SSID '].iter_rows(min_row=1, max_col=5, max_row=75, values_only=True): print(row)





        print ('Excel loaded! :)')

        return excel_document

    except Exception as e:
        pprint("Error loading excel file...")
        pprint(e)
        return (e)




# getOrganizations will return all orgs to which the supplied API key has access
user_choices['all_organizations'] = dashboard.organizations.getOrganizations()

# Prompt the user to pick an organization.
user_choices['organization'] = UserChoice(
    options_list=user_choices['all_organizations'],
    subject_of_choice='organizations',
    single_option_noun='organization'
    )

# Make a list of all networks in the org
user_choices['all_networks'] = dashboard.organizations.getOrganizationNetworks(organizationId=user_choices['organization'].id)

job_order = load_excel(JOB_ORDER_EXCEL_NAME)

info_network = job_order['SSID ']['B3'] # Sede
info_ssid = job_order['SSID ']['B4']
info_ssid_visibility = job_order['SSID ']['B5']
info_ssid_access = job_order['SSID ']['B6']
info_ssid_security = job_order['SSID ']['C8']
info_ssid_password = job_order['SSID ']['C9']
info_ssid_dns = job_order['SSID ']['C34']
info_ssid_fw_rule_policy = job_order['SSID ']['A39']
info_ssid_fw_rule_protocol = job_order['SSID ']['B39']
info_ssid_fw_rule_destination = job_order['SSID ']['C39']
info_ssid_fw_rule_port = job_order['SSID ']['D39']
info_ssid_fw_rule_comment = job_order['SSID ']['E39']

print (info_network.value)
print (info_ssid.value)
print (info_ssid_visibility.value)
print (info_ssid_access.value)
print (info_ssid_security.value)
print (info_ssid_password.value)
print (info_ssid_dns.value)
print (info_ssid_fw_rule_policy.value)
print (info_ssid_fw_rule_protocol.value)
print (info_ssid_fw_rule_destination.value)
print (info_ssid_fw_rule_port.value)
print (info_ssid_fw_rule_comment.value)

# Find the networks with wireless
user_choices['networks_with_wireless'] = [network for network in user_choices['all_networks'] if 'wireless' in network['productTypes']]

# If any are found, let the user choose a network. Otherwise, let the user know that none were found. The logic for this class is defined in a cell above.
user_choices['network'] = UserChoice(
    options_list = user_choices['networks_with_wireless'],
    subject_of_choice = 'networks with wireless',
    single_option_noun = 'network'
    )

#user_choices['network'] = info_network.value

user_data['ssid_info'] = get_ssid_info(networkId = user_choices['network'].id)

ssid_backup_workbook = create_workbook()
add_ssid_info_to_workbook(ssid_backup_workbook, info_network.value)
save_workbook(ssid_backup_workbook)






We found 3 organizations:
0. 454129 with name Telefonica
1. 659777345409781757 with name Blackorg
2. 659777345409783847 with name La Vuelta
Which organization would you like to choose?
Choose 0-2:1
Your organization is Blackorg.
Loading excel...
['Registro', 'GROUP POLICY', 'VLAN', 'SSID ', 'SWITCH', 'SSID completo']
<generator object Worksheet.values at 0x7c4d752e4ac0>
Excel loaded! :)
Blacknet
Merlin
Oculta
Con contraseña WPA2 o WEP
WPA2
Merlin2024
8.8.8.8
ALLOW
ICMP
172.17.0.0/16
Any - Impresión
None
We found 1 networks with wireless:
0. L_659777345409795140 with name Blacknet
Which network would you like to choose?
Choose 0-0:0
Your network is Blacknet.
Created formatted workbook.


KeyError: 'dnsRewrite'